# Calculate Scaling Factors
Andrew E. Davidson  
aedavids@ucsc.edu  
2/23/24  

Copyright (c) 2020-2023, Regents of the University of California All rights reserved. https://polyformproject.org/licenses/noncommercial/1.0.0

Calculates DESeq2 normalization scaling factors

ref: 
- https://bioconductor.org/packages/release/bioc/html/DESeq2.html
-  1. Love, M. I., Huber, W. & Anders, S. Moderated estimation of fold change and dispersion for RNA-seq data with DESeq2. Genome Biol 15, 550 (2014).


In [44]:
import ipynbname
from IPython.display import display
import pandas as pd
# display all rows
#pd.set_option('display.max_rows', None)

# https://joelmccune.com/pandas-dataframe-to-markdown/
#from pandas.io.clipboards import to_clipboard

#import pathlib as pl
import pprint as pp
import numpy as np
import os

In [45]:
dataRoot = "/private/groups/kimlab/GTEx_TCGA/groupbyGeneTrainingSets"
# print(f'!!!!!!!!!! copied data to /scratch/aedavids to speed up testing.')
# dataRoot = "/scratch/aedavids/normalizeCounts.data/"

def loadCSV(path : str) -> pd.DataFrame:
    print(path)
    df = pd.read_csv(path)
    print(df.shape)
    
    return df

#
# create tmp output directory
#
notebookName = ipynbname.name()
# notebookPath = ipynbname.path()
# notebookDir = os.path.dirname(notebookPath)

outDir = f'/scratch/aedavids/{notebookName}.out'
os.makedirs(outDir, exist_ok=True)
print(f'outDir:\n{outDir}')

outDir:
/scratch/aedavids/normalizeCounts.out


In [51]:
%%time
groupbyPath = f'{dataRoot}/GTEx_TCGA_Groupby.csv'
groupbyDF   = loadCSV( groupbyPath )

/private/groups/kimlab/GTEx_TCGA/groupbyGeneTrainingSets/GTEx_TCGA_Groupby.csv
(74777, 26338)
CPU times: user 9min 12s, sys: 32.2 s, total: 9min 44s
Wall time: 9min 44s


In [52]:
def calculateScalingFactor(countDF : pd.DataFrame,
                          verbose : bool = False) -> pd.Series :
    '''
    ref: 
        https://bioconductor.org/packages/release/bioc/html/DESeq2.html 
        1. Love, M. I., Huber, W. & Anders, S. Moderated estimation of fold change and dispersion for RNA-seq data with DESeq2. Genome Biol 15, 550 (2014).

    '''
    # print("\n********** countDF")
    # print(countDF)
    
    # remove rows with zeros. log is undefined
    byRows = 1
    selectRowsWithoutZeros = (countDF != 0).all(axis=byRows)
    if verbose:
        print("\n********** selectRowsWithoutZeros")
        display(selectRowsWithoutZeros)
    
    countDF = countDF.loc[selectRowsWithoutZeros, :]
    if verbose:
        print("\n********** countDF")
        print(countDF)
    
    # natural log
    # smooths over outliers
    logIntensityDF = np.log(countDF)
    if verbose:
        print("\n************ logIntensityDF ")
        display(logIntensityDF)   
    
    # calculate row average of logs. ie geometric means
    # geometric mean is not swayed by outliers. It will 
    # always be <= the arithmetic mean
    rowMeansSeries = logIntensityDF.mean(axis=byRows)
    if verbose:
        print("\n************ rowMeansSeries ")
        display(rowMeansSeries)   
    
    # find metabolites that are signifigantly larger or smaller than 
    # the average subtracting averages from intensities
    # log(a) - log(b) = log(a/b)
    byCols= 0
    outlierDF = logIntensityDF.subtract(rowMeansSeries, axis=byCols)
    if verbose:
        print("\n************ outlierDF ")
        print(type(outlierDF))
        display(outlierDF)   
    
    # calculate the scan/sample median log(estimated scaling factors)
    medianDF = outlierDF.median(axis=byCols)
    if verbose:
        print("\n************ medianDF ")
        print(type(medianDF))
        display(medianDF)      
    
    # convert back to base 10
    extimatedScalingFactorsSeries = np.exp(medianDF)
    # print("\n************ extimatedScalingFactorsSeries ")
    # print(type(extimatedScalingFactorsSeries))
    # display(extimatedScalingFactorsSeries)  
    
    return extimatedScalingFactorsSeries
 

In [53]:
# select the sample columns
sampleCols = ~groupbyDF.columns.isin( ['geneId'] )
countDF = groupbyDF.loc[:, sampleCols]

scalingFactorsSeries = calculateScalingFactor( countDF ) 
print(f'scalingFactorsSeries.shape : {scalingFactorsSeries.shape}' )
print(f'scalingFactorsSeries[0:5] : {scalingFactorsSeries[0:5]}' )

scalingFactorsSeries.shape : (26337,)
scalingFactorsSeries[0:5] : GTEX-1117F-0226-SM-5GZZ7    0.834005
GTEX-1117F-0526-SM-5EGHJ    0.689240
GTEX-1117F-0726-SM-5GIEN    0.537215
GTEX-1117F-2826-SM-5GZXL    0.845562
GTEX-1117F-3226-SM-5N9CT    0.668533
dtype: float64


In [54]:
scalingFactorsPath = f'{dataRoot}/GTEx_TCGA_GroupbyEstimatedSizeFactors.csv'
scalingFactorsSeries.to_csv(scalingFactorsPath, index_label='sampleId', header=['factor'])

In [55]:
aedwip save time do not run test

SyntaxError: invalid syntax (484515467.py, line 1)

## test
The training set scaling factors should match the estimated scaling factors calucalted by DESeq2 [R]
/private/groups/kimlab/GTEx_TCGA/1vsAll/estimatedSizeFactors.csv

In [ ]:
%%time
trainGroupbyPath = f'{dataRoot}/GTEx_TCGA_TrainGroupby.csv'
trainGroupbyDF   = loadCSV( trainGroupbyPath )

In [ ]:
trainGroupbyDF.shape

In [ ]:
%%time 
# select the sample columns
sampleCols = ~trainGroupbyDF.columns.isin( ['geneId'] )
countDF = trainGroupbyDF.loc[:, sampleCols]

scalingFactorsSeries = calculateScalingFactor( countDF ) 
print(f'scalingFactorsSeries.shape : {scalingFactorsSeries.shape}' )
print(f'scalingFactorsSeries[0:5] : {scalingFactorsSeries[0:5]}' )

## Load 
expected scalling factors

In [ ]:
estimatedSizeFactorsPath = f'{dataRoot}/estimatedSizeFactors.csv'
with open(estimatedSizeFactorsPath , 'r') as fd:
    factorStrs = fd.read().splitlines()

print(len(factorStrs))
esfNP = np.array( factorStrs[1:], dtype=float) 
print(esfNP.shape)
esfNP[0:5]
assert sum( np.isclose(scalingFactorsSeries.values, esfNP) ) == scalingFactorsSeries.shape[0], "ERROR "

In [ ]:
# scalingFactorsPath = f'{dataRoot}/testScaling.csv'

# scalingFactorsSeries.to_csv(scalingFactorsPath, index_label='sampleId', header=['factor'])

In [ ]:
groupbyDF.columns[0:5]